## Install OpenAI Library

In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Import Library

In [3]:
import openai
import json
from dotenv import dotenv_values
config = dotenv_values(".env")

## Add API key

In [4]:
openai.api_key  = config["OPENAI_API_KEY"]

## Declare Functions

In [5]:
from IPython.display import Markdown, display

def display_colors1(colors, names):
    display(Markdown(" ".join(
        f'<div style="color: {color}">{name}: {chr(9608) * 5}</div>'
        for color, name in zip(colors, names)
    )))
    
# def display_colors2(colors):
#     display(Markdown(" ".join(
#         f'<div style="color: {color.code}">{color.name}: {chr(9608) * 5}</div>'
#         for color in colors
#     )))
    
def display_colors2(colors):
    display(Markdown(" ".join(
        f'<div style="color: {color["code"]}">{color["name"]}: {chr(9608) * 5}</div>'
        for color in colors
    )))


### 1st Prompt (GPT-3)

In [15]:
def get_and_render_prompt1(text):
    message=f"""
    You are a color palette generating assistant that responds to text prompt for color palettes.
    You should generate color palettes that fit the theme, mood, tone, or instructions in the prompt.
    
    Key Guidelines:
    - Avoid duplicating colors code and name within the same palette.
    - Keep the palettes within a range of 2 to 8 colors.
    - Use a JSON array to represent the color palettes, with each color code in hexadecimal format.
    
    Examples:
    - Q: Convert the following verbal description of a color palette into a list of colors: google brand
      A: ["#4285f4", "#34a853", "#fbbc05", "#ea4335"]

    - Q: Convert the following verbal description of a color palette into a list of colors: midnight pastels
      A: ["#B27DFA", "#6C5DC7", "#3E4AB0", "#2C3E8C", "#182F61", "#0B1E3E", "#080620"]

    Desired Format:
      - a JSON array of hexadecimal color codes

    Prompt: {text}
    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=100,
            temperature=0.8,
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    colors = json.loads(completion.choices[0].text.strip())
    display_colors1(colors)

### 2nd Prompt (GPT-3)

In [7]:
def get_and_render_prompt2(text):
    message=f"""
    Color Palette Generator

    Generate color palettes that match the given theme, mood, or instructions.

    Instructions:
    - Provide a verbal description of the color palette you want.
    - Avoid duplicating colors within the same palette.
    - Keep the palette size between 2 to 8 colors.

    Example Usage:
    - Instruction: Generate a color palette for a Google brand.
    - Output:"["#4285f4", "#34a853", "#fbbc05", "#ea4335"],["Google Blue", "Google Green", "Google Yellow", "Google Red"]"

    - Instruction: Generate a color palette for ocean pastels color palette.
    - Output:"["#83ADFC", "#53CBED", "#22D9DB", "#14D59C", "#0EC36F", "#2EAC8D", "#2A8FBC"], ["Sky Blue", "Light Blue", "Turquoise", "Mint Green", "Emerald Green", "Teal", "Navy Blue"]"

    Desired Response JSON Format:"["#color1", "#color2", ...], ["name1", "name2", ...]"

    Instruction: Generate a color palette for {text}

    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    result = completion.choices[0].text
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors1(colors, names)

### 3rd Prompt (GPT-3.5)

In [8]:
def get_and_render_prompt_chat1(text):
    messages = [
        {
            "role": "system",
            "content": """
            You're Color Palette Generator. 
            You'll generate color palettes that match the given theme, mood, or instructions. 
            You'll avoid duplicating colors and names within the same palette. 
            You'll keep the palette size between 2 to 8 colors.
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for a Google brand."
        },
        {
            "role": "assistant",
            "content": '["#4285f4", "#34a853", "#fbbc05", "#ea4335"], ["Google Blue", "Google Green", "Google Yellow", "Google Red"]'
        },
        {
            "role": "user",
            "content": "Generate a color palette for foresta."
        },
        {
            "role": "assistant",
            "content": '["#3F5F6D", "#2F4F4F", "#1B4F5F", "#006633", "#009933", "#33CC33", "#00CC00"], ["Midnight Blue", "Dark Slate Gray", "Dark Turquoise", "Forest Green", "Jungle Green", "Lime Green", "Green"]'
        },
        {
            "role": "user",
            "content": f"Generate a color palette for {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].message.content)
    print("-----")
    print(completion)

    result = completion.choices[0].message.content.strip()
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors1(colors, names)

In [9]:
### 4th Prompt (GPT-3.5)

In [10]:
def get_and_render_prompt_chat2(text):
    messages = [
        {
            "role": "system",
            "content": """
            You're a colors palette generator assistant who must do your task without adding any comments or notes. 
            You'll generate color palettes that match the given theme, mood, or instructions. 
            You'll avoid duplicating color code or color name within the same response, but you can generate a new palette even if it is the same input.
            You must keep the palette size between 2 to 8 colors.
            You must return a JSON array, where each element follows this format: {"code": <color_code>, "name": <color_name>}
            If you recieve the same input, try generate a new colors palette again.
            If you recieve an empty input, just ramdomly generate a new beautiful colors palette.
            """
        },
        {
            "role": "user",
            "content": "Generate a color palette for: ocean breeze"
        },
        {
            "role": "assistant",
            "content": """
            [
              {"code": "#4ECDC4", "name": "Turquoise"},
              {"code": "#F7FFF7", "name": "Mint Cream"},
              {"code": "#9BC1BC", "name": "Opal"},
              {"code": "#5D5C61", "name": "Slate Gray"}
            ]
            """
        },
        {
            "role": "user",
            "content": f"Generate a color palette for: {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    result = completion.choices[0].message.content
    print(result)
    print("-----")
    print(completion)

    palette = json.loads(result)
    print(palette)
    
    display_colors2(palette)
    

## Insert Your Input

In [14]:
get_and_render_prompt1("ocean breeze")

 ["#A3DEEA", "#56C4E5", "#00AFE2", "#0094C9", "#007AB3", "#006099", "#004B80"]
-----
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " [\"#A3DEEA\", \"#56C4E5\", \"#00AFE2\", \"#0094C9\", \"#007AB3\", \"#006099\", \"#004B80\"]"
    }
  ],
  "created": 1685510675,
  "id": "cmpl-7M8bzd14zSk7HVdpjPm2WtgNAOH01",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 42,
    "prompt_tokens": 265,
    "total_tokens": 307
  }
}


TypeError: display_colors1() missing 1 required positional argument: 'names'

In [12]:
get_and_render_prompt_chat2("foresta")

            [
              {"code": "#228B22", "name": "Forest Green"},
              {"code": "#8F9779", "name": "Camouflage Green"},
              {"code": "#BDB76B", "name": "Dark Khaki"},
              {"code": "#FFF8DC", "name": "Cornsilk"}
            ]
             
(Note: I generated a new palette since the input was the same as the previous one)
-----
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "            [\n              {\"code\": \"#228B22\", \"name\": \"Forest Green\"},\n              {\"code\": \"#8F9779\", \"name\": \"Camouflage Green\"},\n              {\"code\": \"#BDB76B\", \"name\": \"Dark Khaki\"},\n              {\"code\": \"#FFF8DC\", \"name\": \"Cornsilk\"}\n            ]\n             \n(Note: I generated a new palette since the input was the same as the previous one)",
        "role": "assistant"
      }
    }
  ],
  "created": 1685510494,
  "id": "chatcmpl-7M8Z4nwu1QPIoN7uxnYNzWIOGB30F",
  "m

JSONDecodeError: Extra data: line 8 column 1 (char 275)